In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics.cluster import adjusted_rand_score, rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.special import logsumexp
from time import time

from warnings import simplefilter


In [15]:
def get_data():
    data = np.loadtxt('./semeion.data', dtype=np.int8)
    return data[:, :256], data[:, 256:]


def one_hot_decode(y: np.array):
    return np.argmax(y, axis=1)


def get_data_transformed():
    data = np.loadtxt('./semeion.data', dtype=np.int8)
    return data[:, :256], one_hot_decode(data[:, 256:])


In [16]:
# fake class for no dim. red.
class NoDR:
    def __init__(self, n_components):
        pass

    def fit_transform(self, X):
        return X.copy()


In [17]:
simplefilter(action='ignore', category=FutureWarning)
ks = range(5, 16)
ds = (2, 64, 128, 256)


In [18]:
def _log_no_underflow(x):

    return np.log(np.clip(x, 1e-15, 1))

In [19]:
def _estimate_bernoulli_parameters(X, resp):

    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    return nk, means

In [20]:
def _estimate_log_bernoulli_prob(X, means):

    return (X.dot(_log_no_underflow(means).T) +
            (1 - X).dot(_log_no_underflow(1 - means).T))


In [21]:
class Bernoulli_Mixture:
    def __init__(self, n_components, max_iter=500, tol=1e-3):

        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def __str__(self):
        return 'BMM'

    def fit(self, X, init_kmeans=True):

        self.converged_ = False

        n_samples, _ = X.shape
        random_state = 42
        if init_kmeans:
            resp = np.zeros((n_samples, self.n_components))
            label = (
                KMeans(n_clusters=self.n_components, n_init=1, random_state=random_state)
                    .fit(X)
                    .labels_
            )
            resp[np.arange(n_samples), label] = 1
        else:
            resp = random_state.rand(n_samples, self.n_components)
            resp /= resp.sum(axis=1)[:, np.newaxis]

        self._initialize(X, resp)

        lower_bound = -np.inf

        for n_iter in range(1, self.max_iter + 1):
            prev_lower_bound = lower_bound

            log_prob_norm, log_resp = self._e_step(X)
            self._m_step(X, log_resp)
            lower_bound = log_prob_norm

            change = lower_bound - prev_lower_bound

            if abs(change) < self.tol:
                self.converged_ = True
                break

        if not self.converged_:
            raise ValueError('Not converged')

        _, log_resp = self._e_step(X)

        self.labels_ = log_resp.argmax(axis=1)

    def _e_step(self, X):

        log_prob_norm, log_resp = self._estimate_log_prob_resp(X)
        return np.mean(log_prob_norm), log_resp

    def _estimate_weighted_log_prob(self, X):

        return self._estimate_log_prob(X) + self._estimate_log_weights()

    def _estimate_log_prob_resp(self, X):

        weighted_log_prob = self._estimate_weighted_log_prob(X)
        log_prob_norm = logsumexp(weighted_log_prob, axis=1)
        with np.errstate(under="ignore"):
            # ignore underflow
            log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
        return log_prob_norm, log_resp

    def _initialize(self, X, resp):

        n_samples, n_dim = X.shape

        weights, means = _estimate_bernoulli_parameters(X, resp)
        weights /= n_samples

        self.weights_ = weights
        self.means_ = means

    def _m_step(self, X, log_resp):

        n_samples, _ = X.shape
        self.weights_, self.means_ = (
            _estimate_bernoulli_parameters(X, np.exp(log_resp))
        )
        self.weights_ /= n_samples

    def _estimate_log_prob(self, X):
        return _estimate_log_bernoulli_prob(X, self.means_)

    def _estimate_log_weights(self):
        return np.log(self.weights_)


In [22]:
def LCA(p=False):

    columns = ('clustering_model', 'dim_red_model', 'd', 'k', 'time', 'ARI', 'RI')
    rows = []

    # read data and split
    X, y = get_data_transformed()
    # loop clusteting model
    for k in ks:
        for d in ds:
            if d != 256:
                embedding_model = PCA
            else:
                embedding_model = NoDR
            start_time = time()
            embedding = embedding_model(n_components=d)  # DR model
            X_transformed = embedding.fit_transform(X)
            model = Bernoulli_Mixture(k)  # clustering model
            model.fit(X_transformed)
            elapsed = time() - start_time
            title = f'Results for {model} on {d}-{embedding_model.__name__} - k={k} ({elapsed:.02f}s)'
            print(title)
            # RI score
            ari, ri = print_rand(y, model.labels_)
            # plot results
            if p: plot2D(X_transformed, model.labels_, title)
            result = (str(model), embedding_model.__name__, d, k, elapsed, ari, ri)
            rows.append(result)
    # save results to file
    df = pd.DataFrame(data=rows, columns=columns)
    print(df)
    df.to_csv('LCA.csv', index=False)


In [23]:
def print_rand(y_true, y_pred):
    ari = adjusted_rand_score(y_true, y_pred)
    ri = rand_score(y_true, y_pred)
    print('adjusted rand score', round(ari * 100, 2), '%')
    print('rand score', round(ri * 100, 2), '%', end='\n\n')
    return ari, ri

In [24]:
def results_analysis():
    df = pd.read_csv('LCA.csv')


    dfg_k = df.groupby('k').mean()[['time', 'ARI', 'RI']]
    print('averages by number of cluster:')
    print(dfg_k.sort_values('ARI', ascending=False))
    print()

    dfg_dr = df.groupby('dim_red_model').mean()[['time', 'ARI', 'RI']]
    print('averages by DR model:')
    print(dfg_dr.sort_values('ARI', ascending=False))
    print()


    print('best ARI achieved by:')
    print(df.sort_values('ARI', ascending=False).head(3))

In [25]:
def main():
    
    LCA()
    results_analysis()

In [26]:

if __name__ == '__main__':
    main()


Results for BMM on 2-PCA - k=5 (0.06s)
adjusted rand score 16.07 %
rand score 72.41 %

Results for BMM on 64-PCA - k=5 (0.15s)
adjusted rand score 18.72 %
rand score 78.48 %

Results for BMM on 128-PCA - k=5 (0.31s)
adjusted rand score 17.35 %
rand score 78.45 %

Results for BMM on 256-NoDR - k=5 (0.18s)
adjusted rand score 25.26 %
rand score 79.95 %

Results for BMM on 2-PCA - k=6 (0.07s)
adjusted rand score 17.13 %
rand score 75.92 %

Results for BMM on 64-PCA - k=6 (0.19s)
adjusted rand score 24.76 %
rand score 81.9 %

Results for BMM on 128-PCA - k=6 (0.27s)
adjusted rand score 23.66 %
rand score 81.98 %

Results for BMM on 256-NoDR - k=6 (0.66s)
adjusted rand score 30.68 %
rand score 83.59 %

Results for BMM on 2-PCA - k=7 (0.08s)
adjusted rand score 17.06 %
rand score 76.77 %

Results for BMM on 64-PCA - k=7 (0.17s)
adjusted rand score 28.9 %
rand score 84.63 %

Results for BMM on 128-PCA - k=7 (0.33s)
adjusted rand score 26.31 %
rand score 84.1 %

Results for BMM on 256-NoDR - k